In [1]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);


# data import

In [3]:
dispatch_data = CSV.read("zipcode_incident_data.csv", DataFrame)
dispatch_data

Row,ZIPCODE,INCIDENT_COUNT
,Int64,Int64
1,10003,4
2,10027,4
3,10019,3
4,10039,3
5,10035,3
6,10014,3
7,10001,3
8,10025,2
9,10029,2


In [4]:
wait_time = CSV.read("wait_time_matrix.csv",DataFrame,header=false) |> Matrix
wait_time

9×90 Matrix{Float64}:
  2845.0  32387.0  16390.0  32283.0    …     1.0e6     1.0e6      1.0e6
 27984.0  17284.0  28316.0   8441.0          1.0e6     1.0e6      1.0e6
 31549.0   4532.0  28819.0   3305.0       5982.0    9564.0        1.0e6
  1635.0   2760.0   1230.0   1169.0          1.0e6     1.0e6      1.0e6
 31129.0   2727.0   4952.0   9068.0        209.0       1.0e6  13847.0
  1285.0   2347.0   1264.0    706.0    …     1.0e6     1.0e6      1.0e6
  3316.0   1128.0   4032.0   1052.0       1503.0       1.0e6      1.0e6
  4789.0   1038.0   1548.0      1.0e6        1.0e6     1.0e6      1.0e6
  2440.0   1438.0   3092.0   3092.0          1.0e6     1.0e6      1.0e6

In [5]:
ambulance_capacity = CSV.read("area_capacity_data.csv", DataFrame)
ambulance_capacity

Row,Ambulance_i,INCIDENT_DISPATCH_AREA,CAPACITY
,Int64,String3,Int64
1,1,M1,19
2,2,M2,26
3,3,M3,28
4,4,M4,13
5,5,M5,16
6,6,M6,8
7,7,M7,20
8,8,M8,11
9,9,M9,16


In [6]:
zipcode_to_index = CSV.read("zipcode_to_index.csv", DataFrame)
zipcode_to_index

Row,Zipcode,Index
,Int64,Int64
1,10168,69
2,10013,12
3,10048,42
4,10023,21
5,10034,32
6,10014,13
7,10019,17
8,10031,29
9,10038,36


In [7]:
merged_data = leftjoin(zipcode_to_index, dispatch_data, on=:Zipcode => :ZIPCODE)
sorted_dispatch_data = sort(merged_data, :Index)
sorted_dispatch_data


Row,Zipcode,Index,INCIDENT_COUNT
,Int64,Int64,Int64?
1,10001,1,3
2,10002,2,2
3,10003,3,4
4,10004,4,missing
5,10005,5,missing
6,10006,6,missing
7,10007,7,missing
8,10009,8,1
9,10010,9,missing


In [8]:
# convert missing values to 0
sorted_dispatch_data.INCIDENT_COUNT .= coalesce.(sorted_dispatch_data.INCIDENT_COUNT, 0)
sorted_dispatch_data

Row,Zipcode,Index,INCIDENT_COUNT
,Int64,Int64,Int64
1,10001,1,3
2,10002,2,2
3,10003,3,4
4,10004,4,0
5,10005,5,0
6,10006,6,0
7,10007,7,0
8,10009,8,1
9,10010,9,0


## build the model:
3 useful dfs:

dispatch data with sorted index: sorted_dispatch_data   d_j

ambulance capacity: ambulance_capacity c_i 

wait time : wait_time  waittime(i,j)

In [9]:
capacity_vector = ambulance_capacity[!,"CAPACITY"]

9-element Vector{Int64}:
 19
 26
 28
 13
 16
  8
 20
 11
 16

In [10]:
demand_vector = sorted_dispatch_data[!,"INCIDENT_COUNT"]

90-element Vector{Int64}:
 3
 2
 4
 0
 0
 0
 0
 1
 0
 2
 0
 1
 3
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

# Min-Max


Different with Min-Sum, The model introduce a binary variable zij, and also the big M.

T = max{wait_ij * z_ij}

S_{i,j} = 0 means  Z_{i,j} = 0 ; S_{i,j} >=1 means Z_{i,j} = 1.

In [11]:
num_areas = 9
num_zipcodes = 90

# Build the optimization model
model = Model(Gurobi.Optimizer)

# Decision variables: S[i, j] ∈ integer >=0
@variable(model, S[1:num_areas, 1:num_zipcodes] >= 0, Int)

# New binary variable z[i, j] # S_{i,j} = 0 means  Z_{i,j} = 0 ; S_{i,j} >=1 means Z_{i,j} = 1.
@variable(model, z[1:num_areas, 1:num_zipcodes], Bin) 

# New variable T to represent the maximum wait time
@variable(model, T >= 0)

# Objective: Minimize T (the maximum wait time)
@objective(model, Min, T)

# Constraints:

#
# Link z[i, j] and S[i, j] # # S_{i,j} = 0 means  Z_{i,j} = 0 ; S_{i,j} >=1 means Z_{i,j} = 1. here 1e6 is the big M
@constraint(model, [i in 1:num_areas, j in 1:num_zipcodes], S[i, j] <= z[i, j] * 1e6)
@constraint(model, [i in 1:num_areas, j in 1:num_zipcodes], S[i, j] >= z[i, j])

# Ensure T is at least as large as z_ij * wait_time[i, j] for all (i, j) T = max{wait_ij * z_ij}
@constraint(model, [i in 1:num_areas, j in 1:num_zipcodes], T >=  wait_time[i, j] * z[i, j])

# Capacity constraints for each ambulance station
@constraint(model, [i in 1:num_areas], sum(S[i, j] for j in 1:num_zipcodes) <= capacity_vector[i])

# Meet the demand for each zipcode
@constraint(model, [j in 1:num_zipcodes], sum(S[i, j] for i in 1:num_areas) >= demand_vector[j])

# Optimize the model
optimize!(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-19
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2529 rows, 1621 columns and 6480 nonzeros
Model fingerprint: 0x8c478b9d
Variable types: 1 continuous, 1620 integer (810 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 2064 rows and 1296 columns
Presolve time: 0.01s
Presolved: 465 rows, 325 columns, 1296 nonzeros
Variable types: 0 continuous, 325 integer (216 binary)
Found heuristic solution: objective 30132.000000
Found heuristic solution: objective 17924.000000
Found heuristic solution: objective 17642.000000
Found heuristic solution: objective 17284

In [12]:
# Check the solution
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found:")
    solution = value.(S)
else
    println("No optimal solution found.")
end

Optimal solution found:


9×90 Matrix{Float64}:
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  -0.0  -0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  3.0   2.0   4.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [13]:
nonzero_indices = findall(x -> x != 0, solution)  
nonzero_values = solution[nonzero_indices]   

println("nonzero indices: ", nonzero_indices)
println("nonzero values: ", nonzero_values)
println("number of S(i,j)>0:  ", length(nonzero_values))


nonzero indices: CartesianIndex{2}[CartesianIndex(9, 1), CartesianIndex(9, 2), CartesianIndex(9, 3), CartesianIndex(3, 8), CartesianIndex(3, 10), CartesianIndex(8, 12), CartesianIndex(3, 13), CartesianIndex(1, 14), CartesianIndex(1, 15), CartesianIndex(7, 16), CartesianIndex(2, 17), CartesianIndex(2, 23), CartesianIndex(1, 24), CartesianIndex(2, 25), CartesianIndex(1, 27), CartesianIndex(1, 29), CartesianIndex(1, 30), CartesianIndex(1, 32), CartesianIndex(1, 33), CartesianIndex(1, 34), CartesianIndex(2, 36), CartesianIndex(1, 37), CartesianIndex(1, 43), CartesianIndex(1, 50)]
nonzero values: [3.0, 2.0, 4.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 2.0, 3.0, 2.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 3.0, 1.0, 1.0]
number of S(i,j)>0:  24


In [14]:
assignment_matrix = zeros(Int, num_areas, num_zipcodes)

for i in 1:num_areas
    for j in 1:num_zipcodes
        if solution[i, j] > 0
            assigned_count = Int(solution[i, j])
        else
            assigned_count = 0
        end
        assignment_matrix[i, j] = assigned_count
    end
end


In [15]:
assignment_matrix


9×90 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  2  0  0  3     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 3  2  4  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [16]:
column_names = [string("Zipcode_", j) for j in 1:num_zipcodes]
row_names = [string("Area_M", i) for i in 1:num_areas]

df = DataFrame(assignment_matrix, Symbol.(column_names))
df.Area = row_names

CSV.write("MinMax_ambulance_assignments_matrix_named.csv", df)
println("Matrix saved to MinMax_ambulance_assignments_matrix_named.csv.")


Matrix saved to MinMax_ambulance_assignments_matrix_named.csv.


# Calculate Cost Matrixs

1. cost of a region (number of incidents * waittime)

2. cost of single incident

1. The following code is the cost(waittime*assignment) matrix.

In [17]:
cost_matrix = assignment_matrix .* wait_time
df_cost_matrix = DataFrame(cost_matrix, Symbol.(column_names))
df_cost_matrix.Area = row_names
CSV.write("MinMax_cost_matrix_sijWaitij.csv", df_cost_matrix)
println("cost Matrix saved to MinMax_cost_matrix_sijWaitij.csv.")


cost Matrix saved to MinMax_cost_matrix_sijWaitij.csv.


2. The following code is the cost of single incident